# Pydantic
framework to defining specialized python classes  
classes attributes a called fields  
specialized classe is called model  
simple way to load data from a dic or json into a model (python class) - deserializing data  
simple way to extract model instance data to dic or json - serializing model instance  
validation data during deserialization  
pydantic v2 is written in rust  
data classes not provide deserialization and serialization, but it is fast than pydantic   


# Dataclasses
is a code generator, create a brand new class with all the functionality

In [1]:
from dataclasses import dataclass

In [2]:
@dataclass
class Model:
    language: str
    version: str
    year: int

In [3]:
m_dataclass = Model(language="eng", version="1.0", year=2024)

# Pydantic

In [4]:
from pydantic import BaseModel, ValidationError

In [5]:
class ModelPydantic(BaseModel):
    language: str
    version: str
    year: int

In [6]:
m_pydantic = ModelPydantic(language="eng", version="1.0", year=2024)

# Creating a Pydantic Model

In [26]:
class Person(BaseModel):
    first_name: str
    last_name: str
    age: int

    @property
    def full_name(self):
        return f'{self.first_name} {self.last_name}'

In [27]:
p = Person(first_name='Marcos', last_name='Liger', age=23)

In [9]:
str(p)

"first_name='Marcos' last_name='Liger' age=23"

In [10]:
repr(p)

"Person(first_name='Marcos', last_name='Liger', age=23)"

In [11]:
p.model_fields

{'first_name': FieldInfo(annotation=str, required=True),
 'last_name': FieldInfo(annotation=str, required=True),
 'age': FieldInfo(annotation=int, required=True)}

In [12]:
p.full_name

'Marcos Liger'

### By default pydantic not validate when change a field

In [13]:
p.age = 'asdf'

In [14]:
p

Person(first_name='Marcos', last_name='Liger', age='asdf')

# Deserialization

In [15]:
data = {
    'first_name': 'Carlos',
    'last_name': 'Magno',
    'age': 83
}

### Not deserialization, using kwargs expand

In [16]:
cm = Person(**data)
cm

Person(first_name='Carlos', last_name='Magno', age=83)

### Using deserialization with dic

In [17]:
cm = Person.model_validate(data)
cm

Person(first_name='Carlos', last_name='Magno', age=83)

In [18]:
data_missing = {
    'first_name': 'Carlos',
    'last_name': 'Magno'
}

In [19]:
try:
    cm = Person.model_validate(data_missing)
except ValidationError as ex:
    print(ex)

1 validation error for Person
age
  Field required [type=missing, input_value={'first_name': 'Carlos', 'last_name': 'Magno'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/missing


### Using deserialization with json

In [20]:
data_json = '''{
    "first_name": "Carlos",
    "last_name": "Magno",
    "age": 83
}'''

In [21]:
cm = Person.model_validate_json(data_json)
cm

Person(first_name='Carlos', last_name='Magno', age=83)

In [30]:
cm.__dict__

{'first_name': 'Carlos', 'last_name': 'Magno', 'age': 83}

# Serialization

### Serializing to dict

In [32]:
cm.model_dump()

{'first_name': 'Carlos', 'last_name': 'Magno', 'age': 83}

In [34]:
type(cm.model_dump())

dict

### Serializing to json

In [33]:
cm.model_dump_json()

'{"first_name":"Carlos","last_name":"Magno","age":83}'

In [35]:
type(cm.model_dump_json())

str

In [37]:
cm.model_dump_json(indent=2)

'{\n  "first_name": "Carlos",\n  "last_name": "Magno",\n  "age": 83\n}'

In [38]:
print(cm.model_dump_json(indent=2))

{
  "first_name": "Carlos",
  "last_name": "Magno",
  "age": 83
}


### Exclude or include some fields

### Exclude or Include some fields

In [39]:
cm.model_dump(exclude=['first_name'])

{'last_name': 'Magno', 'age': 83}

In [40]:
cm.model_dump(include=['first_name'])

{'first_name': 'Carlos'}

## Type Coercion

In [1]:
from pydantic import BaseModel, ValidationError

In [2]:
class Coordinates(BaseModel):
    x: float
    y: float

In [4]:
p2 = Coordinates(x=1.1, y=2.2)

In [5]:
p2 

Coordinates(x=1.1, y=2.2)

In [6]:
Coordinates.model_fields

{'x': FieldInfo(annotation=float, required=True),
 'y': FieldInfo(annotation=float, required=True)}

In [7]:
p2=Coordinates(x=0, y='1.2')

In [8]:
p2

Coordinates(x=0.0, y=1.2)

In [9]:
type(p2.x), type(p2.y)

(float, float)

### Required vs Optional Fields

In [1]:
from pydantic import BaseModel

In [2]:
class Circle(BaseModel):
    center: tuple[int, int]
    radius: int

In [3]:
Circle.model_fields

{'center': FieldInfo(annotation=tuple[int, int], required=True),
 'radius': FieldInfo(annotation=int, required=True)}

In [4]:
class Circle(BaseModel):
    center: tuple[int, int] = (0, 0)
    radius: int

In [5]:
Circle.model_fields

{'center': FieldInfo(annotation=tuple[int, int], required=False, default=(0, 0)),
 'radius': FieldInfo(annotation=int, required=True)}

In [6]:
Circle(radius=3)

Circle(center=(0, 0), radius=3)